In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `C:\Users\okarl\Desktop\diploma-thesis\ntbks`


In [2]:
using JLD
using Statistics
using DataFrames
using StatsModels
using GLM
using ARCHModels
using Random

include("../src/utils/fcn_data.jl");

In [3]:
function RMSE(true_params, parameters, stds)
    true_normalizer = deepcopy(true_params)
    true_normalizer[true_params .== 0] .= 1

    true_normalizer = reshape(true_normalizer, ((1, size(true_normalizer)[1])))
    return mean(sqrt.(((parameters .- true_normalizer).^2 + stds.^2) ./ abs.(true_normalizer)))
end;

# AR2

In [4]:
mod_set_ar = Dict("model" => "AR2", # model name
               "obs" => 3000, # number of observations [3000]
               "burn" => 200, # burn-in period length [200]
               "cali" => [0.2, -0.9], # model calibration; alpha_1, alpha_2
               "cons" => [(-1., 1.), (-1., 1.)], # search constraints
               "fixed_params" => [sqrt(0.1)]) # sigma

# full setup dictionary
setup_ar = Dict("mod" => mod_set_ar) # model setup

TRUE_AR2 = [0.2, -0.9]

estimates_ar = zeros(100, 2)
stds_ar = zeros(100, 2)
confints_ar = zeros(100, 4);

In [5]:
for i in 1:100
    seed = i
    Random.seed!(seed)
    
    data = generate_series(setup_ar, setup_ar["mod"]["obs"], setup_ar["mod"]["burn"], setup_ar["mod"]["cali"]);
    
    dataset = DataFrame(y=data, y_lag1=[missing; data[1:end-1]], y_lag2=[missing; missing; data[1:end-2]])
    dataset = dropmissing(dataset)
    
    model = lm(@formula(y ~ -1 + y_lag1 + y_lag2), dataset)  # without intercept
    
    estimates_ar[i, :] = coef(model)
    stds_ar[i, :] = stderror(model)
    confints_ar[i, 1:2] = confint(model, 0.95)[1, :]
    confints_ar[i, 3:4] = confint(model, 0.95)[2, :]
end

In [6]:
RMSE(TRUE_AR2, mean(estimates_ar, dims=1), std(estimates_ar, dims=1))

0.013342617366157968

In [7]:
round.(mean(confints_ar, dims=1)[1, :], digits=4)

4-element Vector{Float64}:
  0.1837
  0.2151
 -0.9144
 -0.8831

In [8]:
round.(mean(estimates_ar, dims=1)[1, :], digits=4)

2-element Vector{Float64}:
  0.1994
 -0.8987

In [9]:
jldopen("../results/ols_ar2.jld", "w") do file
    # Save arrays into the file
    file["estimates"] = estimates_ar
    file["stds"] = stds_ar
    file["confints"] = confints_ar
end;

# ARMA(1,1)-GARCH(1,1)

In [10]:
mod_set_garch = Dict("model" => "ARMAGARCH", # model name
               "obs" => 3000, # number of observations [3000]
               "burn" => 200, # burn-in period length [200]
               "cali" => [0.0, 0.7, 0.1, 0.1, 0.3, 0.001], # model calibration; mu, a, b, alpha, beta, omega
               "cons" => [(-1., 1.), (-1., 1.), (-1., 1.), (0., 1.), (0., 1.), (0., 1.)], # search constraints
               "fixed_params" => Float64[])

# full setup dictionary
setup_garch = Dict("mod" => mod_set_garch) # model setup

TRUE_GARCH = [0.0, 0.7, 0.1, 0.1, 0.3, 0.001] #  mu, a, b, alpha, beta, omega

estimates_garch = zeros(100, 6)
stds_garch = zeros(100, 6)
confints_garch = zeros(100, 12);

In [11]:
for i in 1:100
    seed = i
    Random.seed!(seed)
    
    data = generate_series(setup_garch, setup_garch["mod"]["obs"], setup_garch["mod"]["burn"], setup_garch["mod"]["cali"]);
    
    model = fit(GARCH{1, 1}, data; meanspec=ARMA{1,1})

    cs = coef(model)
    ses = stderror(model)
    cis = confint(model, 0.95)
    
    estimates_garch[i, 1], stds_garch[i, 1], confints_garch[i, 1:2] = cs[4], ses[4], cis[4, :]  # nu
    estimates_garch[i, 2], stds_garch[i, 2], confints_garch[i, 3:4] = cs[5], ses[5], cis[5, :]  # a_1
    estimates_garch[i, 3], stds_garch[i, 3], confints_garch[i, 5:6] = cs[6], ses[6], cis[6, :]  # b_1
    estimates_garch[i, 4], stds_garch[i, 4], confints_garch[i, 7:8] = cs[3], ses[3], cis[3, :]  # alpha_1
    estimates_garch[i, 5], stds_garch[i, 5], confints_garch[i, 9:10] = cs[2], ses[2], cis[2, :]  # beta_1
    estimates_garch[i, 6], stds_garch[i, 6], confints_garch[i, 11:12] = cs[1], ses[1], cis[1, :]  # omega
end

In [12]:
RMSE(TRUE_GARCH, mean(estimates_garch, dims=1), std(estimates_garch, dims=1))

0.25821937292150965

In [13]:
round.(mean(confints_garch, dims=1)[1, :], digits=4)

12-element Vector{Float64}:
 -0.0015
  0.0016
  0.665
  0.7339
  0.0525
  0.1515
  0.0533
  0.1486
 -0.1135
  0.6425
  0.0005
  0.0017

In [14]:
round.(mean(estimates_garch, dims=1)[1, :], digits=4)

6-element Vector{Float64}:
 0.0001
 0.6994
 0.102
 0.1009
 0.2645
 0.0011

In [15]:
jldopen("../results/ols_armagarch.jld", "w") do file
    # Save arrays into the file
    file["estimates"] = estimates_garch
    file["stds"] = stds_garch
    file["confints"] = confints_garch
end;